In [54]:
import random
import time
import sys
import collections

In [55]:
# Збільшуємо ліміт рекурсії для великих графів (для DFS)
sys.setrecursionlimit(20000)

### Генерація графів

In [56]:
def generate_graph(n, p):
    """
    Генерує випадковий граф.
    Повертає два представлення: Матрицю суміжності та Список суміжності.
    n: кількість вершин
    p: ймовірність ребра (щільність)
    """
    matrix = [[0] * n for _ in range(n)]
    adj_list = {i: [] for i in range(n)}
    
    for i in range(n):
        for j in range(i + 1, n): # Неорієнтований граф (або орієнтований, якщо прибрати симетрію)
            if random.random() < p:
                # Додаємо ребро i -> j
                matrix[i][j] = 1
                adj_list[i].append(j)
                
                # Для неорієнтованого розкоментуйте наступні рядки:
                matrix[j][i] = 1
                adj_list[j].append(i)
                
    return matrix, adj_list

### Алгоритми

In [57]:
# BFS для Списків суміжності
def bfs_list(adj_list, start_node, n):
    selected = [False] * n
    queue = collections.deque([start_node])
    selected[start_node] = True
    while queue:
        u = queue.popleft()
        for v in adj_list[u]:
            if not selected[v]:
                selected[v] = True
                queue.append(v)

In [58]:
# BFS для Матриці суміжності
def bfs_matrix(matrix, start_node, n):
    selected = [False] * n
    queue = collections.deque([start_node])
    selected[start_node] = True
    while queue:
        u = queue.popleft()
        for v in range(n):
            if matrix[u][v] == 1 and not selected[v]:
                selected[v] = True
                queue.append(v)

In [59]:
# DFS (Рекурсивний) для Списків суміжності 
def dfs_list_recursive(adj_list, u, selected):
    selected[u] = True
    for v in adj_list[u]:
        if not selected[v]:
            dfs_list_recursive(adj_list, v, selected)

In [60]:
# DFS (Рекурсивний) для Матриці суміжності
def dfs_matrix_recursive(matrix, u, selected, n):
    selected[u] = True
    for v in range(n):
        if matrix[u][v] == 1 and not selected[v]:
            dfs_matrix_recursive(matrix, v, selected, n)

### Експеримент та порівняння результатів

In [61]:
def run_experiment():
    # Параметри дослідження 
    sizes = [10, 50, 100, 300, 500] 
    densities = [0.1, 0.5, 0.9] # Рідкі, середні, щільні
    iterations = 20 # Кількість повторів для точності 

    print(f"{'N':<5} | {'P':<5} | {'BFS List (ms)':<15} | {'BFS Matr (ms)':<15} | {'DFS List (ms)':<15} | {'DFS Matr (ms)':<15}")
    print("-" * 80)

    for n in sizes:
        for p in densities:
            graphs = [generate_graph(n, p) for _ in range(iterations)]
            
            # Таймери
            t_bfs_list = 0
            t_bfs_matrix = 0
            t_dfs_list = 0
            t_dfs_matrix = 0
            
            for matrix, adj_list in graphs:
                start_node = 0
                
                # 1. BFS List
                start = time.perf_counter()
                bfs_list(adj_list, start_node, n)
                t_bfs_list += (time.perf_counter() - start)
                
                # 2. BFS Matrix
                start = time.perf_counter()
                bfs_matrix(matrix, start_node, n)
                t_bfs_matrix += (time.perf_counter() - start)
                
                # 3. DFS List (Recursive)
                selected = [False] * n
                start = time.perf_counter()
                dfs_list_recursive(adj_list, start_node, selected)
                t_dfs_list += (time.perf_counter() - start)
                
                # 4. DFS Matrix (Recursive)
                selected = [False] * n
                start = time.perf_counter()
                dfs_matrix_recursive(matrix, start_node, selected, n)
                t_dfs_matrix += (time.perf_counter() - start)

            # Розрахунок середнього часу в мілісекундах
            avg_bfs_list = (t_bfs_list / iterations) * 1000
            avg_bfs_matrix = (t_bfs_matrix / iterations) * 1000
            avg_dfs_list = (t_dfs_list / iterations) * 1000
            avg_dfs_matrix = (t_dfs_matrix / iterations) * 1000
            
            print(f"{n:<5} | {p:<5} | {avg_bfs_list:<15.4f} | {avg_bfs_matrix:<15.4f} | {avg_dfs_list:<15.4f} | {avg_dfs_matrix:<15.4f}")


In [62]:
run_experiment()

N     | P     | BFS List (ms)   | BFS Matr (ms)   | DFS List (ms)   | DFS Matr (ms)  
--------------------------------------------------------------------------------
10    | 0.1   | 0.0065          | 0.0160          | 0.0040          | 0.0138         
10    | 0.5   | 0.0178          | 0.0487          | 0.0369          | 0.0434         
10    | 0.9   | 0.0213          | 0.0491          | 0.0181          | 0.0475         
50    | 0.1   | 0.0650          | 0.4811          | 0.0577          | 0.4873         
50    | 0.5   | 0.1267          | 0.5803          | 0.1363          | 0.5517         
50    | 0.9   | 0.2594          | 0.7862          | 0.2585          | 0.7716         
100   | 0.1   | 0.1537          | 1.9319          | 0.1479          | 1.7953         
100   | 0.5   | 0.2828          | 1.4519          | 0.2934          | 1.4033         
100   | 0.9   | 0.5341          | 1.6431          | 0.5360          | 1.6377         
300   | 0.1   | 0.6111          | 10.8420         | 0.6301 

# Звіт: Порівняльний аналіз алгоритмів BFS та DFS (Варіант 1)

## 1\. Постановка завдання

Метою роботи є реалізація та порівняння ефективності алгоритмів обходу графу:

  * **BFS** (обхід у ширину);
  * **DFS** (обхід у глибину, рекурсивний варіант).

Основна увага приділяється порівнянню часу виконання в залежності від **способу представлення графу** (матриця суміжності проти списків суміжності) на графах різного розміру та щільності.

## 2\. Умови експерименту та методологія

Експеримент проводився з урахуванням наступних вимог:

  * **Розмір графу ($N$):** від 10 до 500 вершин.
  * **Щільність графу ($P$):** 0.1 (розріджені), 0.5 (середні), 0.9 (щільні).
  * **Вимірювання:** Час вимірювався лише для роботи алгоритму, без урахування часу генерації структури даних.
  * **Фільтрація шуму:** Результати усереднено за серією запусків для мінімізації впливу фонових процесів ОС.

### Теоретична складність

Для коректної інтерпретації результатів наведемо теоретичні оцінки:

| Алгоритм | Списки суміжності | Матриця суміжності |
| :--- | :--- | :--- |
| **BFS / DFS** | $O(V + E)$ | $O(V^2)$ |

Де $V$ — кількість вершин, $E$ — кількість ребер.
Для щільних графів $E \approx V^2$, для розріджених $E \ll V^2$.


## 4\. Інтерпретація даних

На основі отриманих даних можна зробити наступні спостереження:

### А. Вплив представлення даних (Matrix vs List)

Це найбільш суттєвий фактор швидкодії.

  * На великих розмірностях ($N=300, 500$) **списки суміжності працюють на порядок швидше**.
  * Це підтверджує теоретичну оцінку: обхід матриці завжди змушує алгоритм перевіряти всі $N$ потенційних сусідів для кожної вершини.

### Б. Вплив щільності графу (P)

  * **Списки суміжності:** Час виконання лінійно зростає зі збільшенням $P$.
      * При $N=500$: час зростає від 1.56 мс ($P=0.1$) до 12.65 мс ($P=0.9$). Це логічно, оскільки $E$ (кількість ребер) зростає, і алгоритм виконує більше ітерацій.
  * **Матриця суміжності:** Час виконання є відносно стабільним і мало залежить від $P$.
      * При $N=300$: час коливається в межах 13-14 мс незалежно від щільності. Це пояснюється тим, що складність $O(V^2)$ залежить лише від розміру матриці, а не від кількості одиниць у ній.

### В. Порівняння BFS та DFS

Суттєвої різниці між ітеративним BFS та рекурсивним DFS не виявлено.

  * Часові показники майже ідентичні (різниця в межах похибки вимірювань).
  * Наприклад, для $N=500, P=0.5$: BFS List = 6.84 мс, DFS List = 6.88 мс.
  * Це свідчить про те, що накладні витрати на рекурсію в Python (для даних розмірностей) співмірні з накладними витратами на обслуговування черги `collections.deque` у BFS.

## 5\. Висновки

1.  Для задач обходу графів (BFS/DFS) **списки суміжності є значно ефективнішою структурою даних**, ніж матриця суміжності, особливо для розріджених графів ($P < 0.5$).
2.  Використання матриці суміжності виправдане лише у випадках, коли граф надзвичайно щільний ($P \to 1$), або коли потрібні специфічні матричні операції (наприклад, алгоритм Флойда-Уоршелла).
3.  Швидкодія BFS та рекурсивного DFS є еквівалентною на практиці для графів розміром до 500 вершин.
